In [1]:
"""
TO DO
 
1. Run kepras mel example with keras datagen class you designed https://medium.com/@ensembledme/writing-custom-keras-generators-fe815d992c5a
2. Add plotly suite for all transformations
3. https://www.youtube.com/watch?v=W_Wbnp_uw-o
"""

'\nTO DO\n \n1. Run kepras mel example with keras datagen class you designed https://medium.com/@ensembledme/writing-custom-keras-generators-fe815d992c5a\n2. Add plotly suite for all transformations\n'

In [158]:
# https://pypi.org/project/pysndfx/
from pysndfx import AudioEffectsChain
import os
import matplotlib.pyplot as plt
import warnings
from librosa import load
import sys
from inspect import getmembers, signature
from pprint import pprint
from flatten_json import flatten
import numpy as np
import random
import keras
import re

%matplotlib inline

warnings.simplefilter('ignore')

DOWNLOAD_PATH = '/Users/BhavishDaswani/Downloads/IDMT-SMT-GUITAR_V2/dataset1/'

# PYSNDFX defaults

In [160]:
def return_global_spec(fx):
    """
    Return pretty repr of default values
    """
    
    def as_dict(sig):
        
        a_place_for_storage = {}
        for param in sig.parameters.values():
            if (param.kind == param.KEYWORD_ONLY and param.default is param.empty):
                pass
            else:
                a_place_for_storage[param.name] = param.default
        return a_place_for_storage
    
    return {k:  as_dict(signature(v)) for k,v in getmembers(fx) if \
           k.endswith('_') is False and 'command' not in k}

test_class = AudioEffectsChain()
# pprint (return_global_spec(test_class))

# DATA AUGMENTATION

In [ ]:
# SAMPLE EFFECTS
effects = {'overdrive' : {'gain' : (None, 75), 'colour' : (None, 20)},
                   'reverb' : {'wet_only' : 'toggle'}      
                  }

# DESIRED GENERATOR INPUT
batch_settings = {
    
    'settings' : effects,
    'exercise' : 'classification',
    'batch_size' : 2, # 30 files
    'generate_n' : 20
    
}




In [151]:


    
raw_files = training_audio.audioFileName

datagen = sound_generator(soundfiles = raw_files, **batch_settings)

# one batch
features, label_array = next(datagen)


['overdrive', 0, 7]
['overdrive', 0, 7, 'reverb', 50, 50, 100, 100, 20, 0]
['overdrive', 2, 0]
['overdrive', 2, 0, 'reverb', 50, 50, 100, 100, 20, 0]
['overdrive', 0, 6]
['overdrive', 0, 6, 'reverb', 50, 50, 100, 100, 20, 0]
['overdrive', 0, 4]
['overdrive', 0, 4, 'reverb', 50, 50, 100, 100, 20, 0]
['overdrive', 29, 9]
['overdrive', 29, 9, 'reverb', 50, 50, 100, 100, 20, 0]
['overdrive', 0, 2]
['overdrive', 0, 2, 'reverb', 50, 50, 100, 100, 20, 0]
['overdrive', 50, 0]
['overdrive', 50, 0, 'reverb', 50, 50, 100, 100, 20, 0]
['overdrive', 0, 0]
['overdrive', 0, 0, 'reverb', 50, 50, 100, 100, 20, 0]
['overdrive', 45, 0]
['overdrive', 45, 0, 'reverb', '-w', 50, 50, 100, 100, 20, 0]
['overdrive', 0, 0]
['overdrive', 0, 0, 'reverb', 50, 50, 100, 100, 20, 0]
['overdrive', 24, 18]
['overdrive', 24, 18, 'reverb', '-w', 50, 50, 100, 100, 20, 0]
['overdrive', 14, 0]
['overdrive', 14, 0, 'reverb', '-w', 50, 50, 100, 100, 20, 0]
['overdrive', 0, 10]
['overdrive', 0, 10, 'reverb', 50, 50, 100, 100, 

In [155]:
features[0], label_array[0]

(array([[ 0.00018311,  0.00018311,  0.00018311, ..., -0.00210857,
         -0.00211167, -0.00210929]]), array([(1., 0., 0.)],
       dtype=(numpy.record, [('overdrive.colour', '<f8'), ('overdrive.gain', '<f8'), ('reverb.wet_only', '<f8')])))

In [49]:
label_array.shape, features.shape

((10, 1), (10, 1, 110250))

In [18]:
label_array.dtype.names.index('overdrive.gain')

1

In [157]:
label_array['overdrive.gain'].ravel().shape # ravel for one to one relationship

(40,)

In [28]:
features.shape

(10, 1, 110250)

# Plotting toolkit

In [91]:

def get_meta(index): # pull index from dataframe
    return all_data_meta.iloc[index,:]
#     
# def wave_plot():
    
def plot_suite(index, raw_wave = True, m):
    
    
    
    
    
    


SyntaxError: invalid syntax (<ipython-input-91-3c28819132f4>, line 6)

**FOUR LAYERS BY KAPRE - All backprop traininable!**

1. Melspectrogram -  
> extended layer based on Spectrogram with a multiplication by mel-scale conversion
> matrix from linear frequencies which can be trained.
2. Normalization2D
>  normalises 2D input data (e.g., timefrequency representations) per frequency, time, channel,
>(single) data, and batch.
3. Filterbank 
> provides a general filterbank layer that can initialised with mel/log/linear frequency scales as well
> as random.
4. AdditiveNoise 
> adds different types of noise for data augmentation. The noise gain can be randomised for further
> augmentation. Noise is only applied in training p





#  Lets look at what mel does to the shape of our features for insight into desired input shapes



In [86]:
import librosa 

y = features[0].ravel() # cannot mel with monophonic, ravel flattens it across first axis

# optical spectrometer, a bank of band-pass filters or by Fourier transform. https://en.wikipedia.org/wiki/Spectrogram


# Note, youc an pass a precomputed spectogram in lieu of default, this is likely how layers are working iteratively below
# librosa.feature.melspectrogram(y=y, sr = 41000)

librosa_sample = librosa.load(librosa.util.example_audio_file())
sample_y, sample_sr = librosa_sample

def mel(y):
    D = np.abs(librosa.stft(y))**2
    # S = librosa.feature.melspectrogram(S=D)
    S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128, fmax=8000)
    return librosa.power_to_db(S,ref = np.max)

sampled_song_mel = mel(sample_y)
our_sample = mel(y)

# plt.figure(figsize=(10,8))
# plt.subplot(3,1,1)
# librosa.display.specshow(mel(sample_y), y_axis = 'mel', x_axis = 'time')
# plt.suptitle('Librosa Sample')
# plt.subplot(3,1,2)
# librosa.display.specshow(mel(y), y_axis = 'mel', fmax = 8000, x_axis = 'time')
# plt.title('Mel spectrogram')    
# plt.tight_layout()



In [87]:
(sampled_song_mel.shape,our_sample.shape)

((128, 2647), (128, 216))

# Training the model

In [147]:
label_array['overdrive.gain']

array([[1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],

In [143]:
def one_hot_encode(labels):
    n_labels = len(labels)
    n_unique_labels = len(np.unique(labels))
    one_hot_encode = np.zeros((n_labels, n_unique_labels))
    one_hot_encode[np.arange(n_labels), labels] = 1
    return one_hot_encode



In [148]:
# from sklearn.model_selection import train_test_split
# from keras.models import Sequential
# from kapre.time_frequency import Melspectrogram
# from kapre.utils import Normalization2D
# from kapre.augmentation import AdditiveNoise
# from kapre.time_frequency import Spectrogram

from keras import models
from keras import layers
from keras import backend as K

K.clear_session()
# Spectrogram().
model = Sequential()
# network.add(layers.Dense(512, activation='relu', input_shape=(28 * 28,)))
# network.add(layers.Dense(1, activation='softmax'))

# model takes(channel, length as input shape)
# model.add(Spectrogram(n_dft=512, n_hop=256, input_shape=features.shape[1:], 
#           return_decibel_spectrogram=False, power_spectrogram=2.0, 
#           trainable_kernel=True, name='trainable_stft')) 
model.add(Spectrogram(512, input_shape=(features.shape[1:])))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation = 'relu'))
model.add(layers.Dense(1, activation = 'sigmoid'))
# model.compile('adam', loss = 'mse', metrics = ['mse'])
model.compile('rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.summary()

# labels = 

model.fit(features, label_array['overdrive.gain'].ravel(), epochs = 5, batch_size = 32)



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
spectrogram_1 (Spectrogram)  (None, 257, 431, 1)       263168    
_________________________________________________________________
flatten_1 (Flatten)          (None, 110767)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                7089152   
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 7,352,385
Trainable params: 7,089,217
Non-trainable params: 263,168
_________________________________________________________________
Epoch 1/5
600/600 [==============================] - 10s 17ms/step - loss: 6.8276 - acc: 0.5117
Epoch 2/5
600/600 [==============================] - 10s 17ms/step - loss: 7.1716 - acc: 0.5367
Epoch 3/5
600/600 [==============================] - 10s 17ms/step